In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import optimizers

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model 

import pickle
import time

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/data_2categ/dataset_augmented/train",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/data_2categ/dataset_augmented/train",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)


Found 7871 images belonging to 2 classes.
Found 1966 images belonging to 2 classes.


In [4]:
vgg = VGG16(include_top=False, weights='imagenet', input_shape = (224,224, 3))
vgg.get_layer('block1_conv1').trainable = False
vgg.get_layer('block1_conv2').trainable = False
vgg.get_layer('block2_conv1').trainable = False
vgg.get_layer('block2_conv2').trainable = False
vgg.get_layer('block3_conv1').trainable = False
vgg.get_layer('block3_conv2').trainable = False
vgg.get_layer('block3_conv3').trainable = False

for layer in vgg.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f6c1853aa20> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6c18503ac8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6c1853e1d0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6c1853e630> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6c184bf438> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb64cdb00> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6bb52601d0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb5260a90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb47c5908> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb47c59b0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6bb47cb908> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f

In [5]:
NAME = "dimentia-simple-CNN-{}".format(int(time.time()))

model = Sequential()

model.add(vgg)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
#model.add(BatchNormalization())

model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))
    
model.summary ()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3211392   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [6]:
tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/CNN_pretrained/logs/{}".format(NAME))

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)
#filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'],
              )


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[es, checkpoint, tensorboard])


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
  1/245 [..............................] - ETA: 0s - loss: 0.8632 - accuracy: 0.5625WARNING:tensorflow:From /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
216/245 [=========================>....] - ETA: 11s - loss: 0.7124 - accuracy: 0.4957

KeyboardInterrupt: 

In [11]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

[0.6931490898132324, 0.5015369057655334]